In [ ]:
import mne
import sys

from functions.detect_bad import bad_channels

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

In [ ]:
def preprocess(s: int):
    
    # Read data
    ses1 = dirs['eeg'] + '/eeg_s' + str(s) + 'a.bdf'
    ses2 = dirs['eeg'] + '/eeg_s' + str(s) + 'b.bdf'

    raw1 = mne.io.read_raw_bdf(ses1, preload=True)
    raw2 = mne.io.read_raw_bdf(ses2, preload=True)

    raw = mne.concatenate_raws([raw1, raw2])

    # Re-reference
    raw.set_eeg_reference(channels['ref'])

    # Bipolar emg/eog
    raw = mne.set_bipolar_reference(raw, 'EXG3', 'EXG4', 'EOG',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG5', 'EXG6', 'EMGL',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG7', 'EXG8', 'EMGR',
                                    copy = False, drop_refs = True)

    raw.set_channel_types({'EOG':'eog'})
    raw.set_channel_types({'EMGL':'emg', 'EMGR':'emg'})

    # Drop and interpolate
    raw.drop_channels(channels['drop'])

    raw = bad_channels(raw, picks = "eeg", sig_lev=0.05)
    raw.interpolate_bads()

    # High pass filter EMG
    raw.filter(l_freq = 60, h_freq = None, picks = ['EMGL','EMGR'])

    # Resample
    raw.resample(200)

    # Montage
    biosemi64 = mne.channels.make_standard_montage('biosemi64')
    eeg_map = dict(zip(raw.info.ch_names[:64], biosemi64.ch_names))

    raw.rename_channels(eeg_map)
    raw.set_montage(biosemi64)

    # Save clean raw
    raw_fname = dirs['raw'] + '/raw_s' + str(s) + '.fif'
    
    raw.save(raw_fname, overwrite = True)

    return raw

In [ ]:
s = 1

raw = preprocess(s)

raw.info